# Hyperparameter Optimization
本笔记参考了邱锡鹏[《神经网络与深度学习》](https://nndl.github.io)、[知乎文章](https://zhuanlan.zhihu.com/p/51094912)

无论是机器学习还是深度学习模型中，都不可避免的有许多超参数。这些超参数对网络性能的影响还非常的大，所以，为模型选取合适的超参数是提高模型性能的有效途径。

常见的超参数分为如下几类
1. 网络结构, 包括神经元之间的连接关系、层数、每层的神经元数量、激活函数的类型

2. 优化参数, 包含优化方法、学习率、批大小等数据

3. [正则化](../ml/regularization.ipynb)系数

优化的方法有如下几种

## 暴力求解的方法
### Grid Search
网格搜索它的想法很淳朴，把所有的超参数组合尝试一遍后选择最优的超参数组合。

正因为其淳朴的思想，算法的时间复杂度岁超参数的个数指数增加

### Random Search 
不同超参数对模型性能的影响有很大差异. 有些超参数对模型性能的影响有限，而另一些超参数(比如学习率)对模型性能影响比较大. 
在这种情况下，采用网格搜索会在不重要的超参数上进行不必要的尝试. 一种在实践中比较有效的改进方法是对超参数进行随机组合，
然后选取一个性能最好的配置.
随机搜索在实践中更容易实现，一般会比网格搜索更加有效.

## Bayesian Optimization

贝叶斯优化是一种自适应的超参数优化方法，根据当前已经试验的超参数组合，来预 测下一个可能带来最大收益的组合.

一种比较常用的贝叶斯优化方法为时序模型优化. 假设超参数优化的函数$f(x)$服从[高斯过程](https://zhuanlan.zhihu.com/p/56562456)，则$p(f(x)|x)$为一个正态分布. 贝叶斯优化过程是根据已有的$N$组试验结果$\mathcal{H} = \{ x_n, y_n\}_{n = 1}^N$($y_n$为$f(x_n)$的观测值)来建模高斯过程，并计算$f(x)$的后验分布$P(f(x)|x, \mathcal{H})$

为了使得 $P(f(x)|x, \mathcal{H})$接近其真实分布，就需要对样本空间进行足够多 的采样. 但是超参数优化中每一个样本的生成成本很高，需要用尽可能少的样本 来使得$P_{\theta}(f(x)|x, \mathcal{H})$ 接近于真实分布. 因此，需要通过定义一个收益函数(Acquisition Function)$a(x, \mathcal{H})$来判断一个样本是否能够给建模$P_{\theta}(f(x)|x, \mathcal{H})$提 供更多的收益. 收益越大，其修正的高斯过程会越接近目标函数的真实分布.

收益函数的定义有很多种方式. 一个常用的是期望改善(Expected Improvement，EI)函数. 假设$y^* = \min\{y_n, 1\le n \le N\}$是当前已有样本中的最优值，期望改善函数为


$$
\text{EI}(x, \mathcal{H}) = \displaystyle\int_{-\infty}^{\infty}\max(y^* - y,\ 0)P(f(x)|x, \mathcal{H})dy\tag{1}
$$


期望改善是定义一个样本$x$在当前模型$P(f(x)|x, \mathcal{H})$下, $f(x)$超过最好结果$y^*$的期望. 除了期望改善函数之外，收益函数还有其他定义形式，比如改善概 率(Probability of Improvement)、高斯过程置信上界(GP Upper Confidence Bound，GP-UCB)等.

时序模型优化过程如下

>Input: 优化目标函数$f(f)$,迭代次数T,收益函数$a(x, \mathcal{H})$
>
>1. $\mathcal{H} \gets \varnothing$
>2. 随机初始化高斯过程， 并计算$P(f(x)|x, \mathcal{H})$
>3. for $t\gets 1$ to T do
>   1. $x'\gets \arg\underset{x}{\max} a(x, \mathcal{H})$
>   2. 评价$y' = f(x')$
>   3. $\mathcal{H}\gets \mathcal{H}\cup (x', y')$
>   4. 根据$\mathcal{H}$重新计算高斯过程， 并计算$P(f(x)|x, \mathcal{H})$
>4. end
>
>Output: $\mathcal{H}$


贝叶斯优化的一个缺点是高斯过程建模需要计算协方差矩阵的逆，时间复杂度是$O(N^3)$，因此不能很好地处理高维情况. 深度神经网络的超参数一般比 较多，为了使用贝叶斯优化来搜索神经网络的超参数，需要一些更高效的高斯 过程建模. 也有一些方法可以将时间复杂度从$O(N^3)$降低到$O(N)$.